In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/aneesh10/cricket-shot-dataset", force=True)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: akshayhedaoo
Your Kaggle Key: ··········


100%|██████████| 645M/645M [00:27<00:00, 24.5MB/s]


In [ ]:
import torch
from torchvision import transforms

import matplotlib.pyplot as plt
import cv2
import numpy as np

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO
import cv2
# Load a model
model = YOLO('./yolov8m-pose.pt')  # load a custom model


In [ ]:
def get_points(path):
  img = cv2.imread(path)
  img = cv2.resize(img, (640, 640))
  results = model(img, verbose = False)  # predict on an image
  keypoints = results[0].keypoints  # Keypoints object for pose outputs
  result = keypoints.xy.detach().cpu().numpy()[0].reshape(-1).tolist()
  return result

In [ ]:
import pandas as pd
from glob import  glob

imgs = []
cls = []
for x in glob("cricket-shot-dataset/data/*/*"):
  imgs.append(x)
  cls.append(x.split("/")[-2])

In [ ]:
len(imgs), len(cls)

(4482, 4482)

In [ ]:
df = pd.DataFrame(columns = ([x for x in range(0, 34)]))
df

,0,1,2,3,4,...,29,30,31,32,33


In [ ]:
for x in range(len(imgs)):
  try:
    pts = get_points(imgs[x])
    new_pts = " ".join([str(np.round(x, 2)) for x in pts])

    c = cls[x]
    with open("data.csv", "a") as f:
      f.write(f"{new_pts} {c}\n")
    del pts, new_pts
  except Exception as e:
    print(x, e)
    continue


  # del pts, new_pts

In [ ]:
a = get_points("/content/drive/MyDrive/yolov7/cricket-shot-dataset/data/legglance-flick/legglance1002.png")


0: 640x640 2 persons, 11.1ms
Speed: 3.2ms preprocess, 11.1ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
columns = [str(x) for x in range(0, 34)]
columns.append("labels")
columns

In [ ]:
import pandas as pd
df = pd.read_csv("./data.csv", delimiter = " ")
df.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,labels
0,234.92,73.95,242.82,65.81,237.48,67.52,262.39,78.46,251.32,80.79,...,317.61,282.81,413.99,186.65,390.30,329.05,567.26,180.33,545.94,drive
1,454.27,144.41,450.30,130.03,447.73,135.88,423.21,125.67,414.84,135.49,...,350.20,422.66,471.98,305.60,496.95,375.53,599.76,186.03,601.11,drive
2,453.15,144.14,449.82,129.99,446.40,134.75,423.68,126.60,413.40,134.27,...,352.96,424.09,473.54,302.76,498.43,378.24,599.20,184.47,604.85,drive
3,460.23,397.86,452.53,399.77,467.26,399.10,437.77,390.18,469.21,390.83,...,281.07,336.61,318.17,335.18,327.87,274.21,336.24,278.84,338.59,drive
4,330.16,123.68,342.00,110.33,316.14,110.18,350.79,111.98,292.28,111.17,...,333.48,400.51,426.21,184.13,439.86,319.45,556.62,82.24,576.05,drive


In [ ]:
df["labels"].value_counts()

pullshot           1086
legglance-flick    1064
sweep              1018
drive               857
Name: labels, dtype: int64

In [ ]:
df.shape

(4482, 35)

In [ ]:
df.dropna(inplace = True)

In [ ]:
df.shape

(4025, 35)

In [ ]:
df["labels"].value_counts()

pullshot           1086
legglance-flick    1064
sweep              1018
drive               857
Name: labels, dtype: int64

In [ ]:
df["labels"] = df["labels"].map({"pullshot" : 0, "legglance-flick" : 1, "sweep": 2, "drive": 3})

In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,labels
0,234.92,73.95,242.82,65.81,237.48,67.52,262.39,78.46,251.32,80.79,...,317.61,282.81,413.99,186.65,390.30,329.05,567.26,180.33,545.94,3
1,454.27,144.41,450.30,130.03,447.73,135.88,423.21,125.67,414.84,135.49,...,350.20,422.66,471.98,305.60,496.95,375.53,599.76,186.03,601.11,3
2,453.15,144.14,449.82,129.99,446.40,134.75,423.68,126.60,413.40,134.27,...,352.96,424.09,473.54,302.76,498.43,378.24,599.20,184.47,604.85,3
3,460.23,397.86,452.53,399.77,467.26,399.10,437.77,390.18,469.21,390.83,...,281.07,336.61,318.17,335.18,327.87,274.21,336.24,278.84,338.59,3
4,330.16,123.68,342.00,110.33,316.14,110.18,350.79,111.98,292.28,111.17,...,333.48,400.51,426.21,184.13,439.86,319.45,556.62,82.24,576.05,3


In [ ]:
target = df.pop("labels")
data = df

In [ ]:
print(data.shape)
print(target.shape)

(4025, 34)
(4025,)


In [ ]:
from sklearn.preprocessing import StandardScaler
norm = StandardScaler()
data = norm.fit_transform(data)
data.shape

(4025, 34)

In [ ]:
data

In [ ]:
# Train Test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.15, random_state = 0)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(3421, 34)
(3421,)
(604, 34)
(604,)


In [ ]:
# SVM with Linear and RBF kernel
from sklearn.svm import SVC
import numpy as np
from sklearn.model_selection import GridSearchCV
tuned_parameters = [{'kernel':['linear'], 'C':[1, 10, 100, 1000]},
                    {'kernel':['rbf'], 'gamma':np.random.rand(3), 'C': [1, 10, 100, 1000]}]
SVM = GridSearchCV(SVC(), tuned_parameters, scoring='accuracy')
SVM.fit(X_train, y_train)
print('Best Parameters in SVC are :\n')
print(SVM.best_estimator_)
print(SVM.best_params_)
print(SVM.score(X_test, y_test))

Best Parameters in SVC are :

SVC(C=10, gamma=0.09497702028480148)
{'C': 10, 'gamma': 0.09497702028480148, 'kernel': 'rbf'}
0.9039735099337748


In [ ]:
svm = SVC(C= 10, gamma= 0.09497702028480148, kernel= 'rbf')
svm.fit(X_train, y_train)

SVC(C=10, gamma=0.09497702028480148)

In [ ]:
import joblib
joblib.dump(svm, "./model.pkl")

In [ ]:
svm = joblib.load("./model.pkl")

In [ ]:
import warnings
warnings.filterwarnings("ignore")
classes = ["pullshot", "legglance-flick", "sweep", "drive"]
a = get_points("/content/images.jpg")
classes[svm.predict(norm.transform([a]))[0]]

'pullshot'

In [ ]:
! python predict.py -p ./legflick.jpg

legglance-flick
